# **A) Install NLPPie library**

###1.   Installation (v0.3.6)


In [ ]:
pip install nlp-pie==0.3.6

  Using cached https://files.pythonhosted.org/packages/74/6a/4f15285961d2859398e23eea4ca2e79f055f3468b4489d7ffa218bcf86b8/nlp_pie-0.3.6-py2.py3-none-any.whl


###2.   Set the path to **/Pie** folder (usually '/usr/local/lib/python3.6/dist-packages/pie' in GColab if using Python 3.*)

In [ ]:
import os

def set_path_Pie():
  """ Set path to Pie folder for training or tagging
  """
  if not str(os.getcwd()).endswith('/pie'):
    os.chdir(pie_path)
    print(f"Path set to : {os.getcwd()}")
  else:
    pass

In [ ]:
pie_path = '/usr/local/lib/python3.6/dist-packages/pie'
set_path_Pie()

## **B) Training**

In [ ]:
## MODIFIER LES PARAMETRES ET OPTIONS NECESSAIRES (*)

defaultSettingsFile = "default_settings.json" # // name or path to the .json file training parameters *

out_json = defaultSettingsFile.split(".")[0]+"_"+data_to_train+".json"

data_to_train = 'pos' # // train pos (traning data column name) *
modelname = "oldfrench_model" # // output model name *
inputpath = "token_tags_corpus.csv" # // input trainig data path or filename *
colnames = f'["token", "{data_to_train}"]' # // column names in trainig data

customized_train_params = True # // If True the folowing parameters will be used * :
nb_epoch = 50 # *
checks_per_epoch = 0  # *
batch_size = 2  # *
lower_opt = 'false' # lowercase target tokens *

In [ ]:
import json
import pandas as pd
import re

def setParams():
  """Définir les paramètres et les options dans 'default_settings.json'
  """
  with open(defaultSettingsFile, "r") as inp, open(out_json, "w") as out:
      for line in inp:
        if '"modelname"' in line:
          line = line.replace('"model"', f'"{modelname}"')
          print(line)
        if '"input_path"' in line:
          line = line.replace('""', f'"{inputpath}"')
          print(line)
        if '"tasks_order":' in line:
          line = line.replace('["lemma", "pos"]',f'{colnames}')
          print(line)
        if '"name": "lemma"' in line:
          line = line.replace('"lemma"',f'"{data_to_train}"')
          print(line)
        
        if customized_train_params == True:
          if '"epochs"' in line:
            line = re.sub('[0-9]*\,', str(nb_epoch)+",", line)
            print(line)
          if '"batch_size"' in line:
            line = re.sub('[0-9]*\,', str(batch_size)+",", line)
            print(line)
          if '"checks_per_epoch"' in line:
            line = re.sub('[0-9]*\,', str(checks_per_epoch)+",", line)
            print(line)
          if '"lower"' in line:
            line = line.replace('true',lower_opt)
            print(line)

        out.write(line)

In [ ]:
#Apperçu des paramètres
print(f"Parameters : \n{out_json}")
print()
setParams()

*   **Run training & download model after training**



In [ ]:
# Run after setting trainig parameters in the default settings json file & upload training data
!pie train default_settings_pos.json

In [ ]:
from google.colab import files
import glob
lastmodel = modelname+"*.tar"
get_model = "".join(glob.glob(lastmodel)) # 1 model
files.download(get_model) ## // telecharger automatiquement le modele sur l'ordinateur

## **C) Tagging**

###1. Log into your Drive to choose the files to be tagged

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###2. Modify *models/tag_pipe.py* to skip tokenization

In [ ]:
def noTokenizing():
  with open("scripts//tag_pipe.py","r") as py, open("scripts//tag_pipe2.py","w") as out_py:
    f = py.readlines()
    for l in f:
      if "line = line.split()" in l:
        l = l.replace("line = line.split()", "line = line.split()\n            line = [' '.join(item for item in line)]")
      out_py.write(l)
def checkPy():
  with open("scripts//tag_pipe.py","r") as py:
    f = py.readlines()
    for l in f:
      if "line = [' '.join(item for item in line)])" in l:
        pass
      else:
        noTokenizing()

*   **Le modifier de forme automatique**



In [ ]:
!mv "scripts//tag_pipe.py" "scripts//tag_pipe_src.py"
!mv "scripts//tag_pipe2.py" "scripts//tag_pipe.py"

In [ ]:
noTokenizing()
checkPy()

* **Où ajouter le code suivant après la ligne 24 dans *models/tag_pipe.py* :**

`line = [' '.join(item for item in line)]`

dans : 
```
23        else:
24            line = line.split()
25            line = [' '.join(item for item in line)]
```



###3. Tagging target files

In [ ]:
### 
# Chemin vers les fichiers avec les tokens a etiqueter
tokens_path = '/content/gdrive/My Drive/RNNTagger(tagger)/tsv_files/tokens/*tokens.csv' 
# Chemin/nom du modele a utiliser
model = '10ep2bat.tar'

* **Étiqueter plusieurs fichiers**



In [ ]:
import pandas as pd
import subprocess
import glob
import csv
import os, sys


In [ ]:
def run_tagging():
  all_files = glob.glob(tokens_path)
  print(f"{len(all_files)} fichiers trouves.")
  nbFile = 0

  for file in all_files:
    print(file)
    outputname = "/content/gdrive/My Drive/tagged_corpus_nlppie/" + file.split("/")[-1].replace('tokens.csv','_tagged(nlppie_10ep2bat).csv')
    command = "cat '" + file + "' | " + "pie tag-pipe " + f"{model} > '" + outputname + "'"
    nbFile += 1
    print(f"Fichier no. : {nbFile}")
    !{command} 

* **Étiqueter un seul fichier**

In [ ]:
inputFile = 'tokens.csv' # // nom du fichier a etiqueter
output_tagged = 'tokens_annotes.txt' # // nom du fichier en sortie

In [ ]:
%%shell
cat {inputFile} | pie tag-pipe {model} > {output_tagged}

fatal: not a git repository (or any of the parent directories): .git
